In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('../../../data/chocolate.csv')

In [3]:
df

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,5,Jacque Torres,U.S.A.,2006,Ghana,Trinatario Treasure,71%,"5- B,S,C,V,L","gritty, unrefined, off notes",2.00
1,15,Neuhaus (Callebaut),Belgium,2006,Sao Tome,Sao Tome,75%,"5- B,S,C,V,L","grassy, earthy, burnt",2.75
2,15,Neuhaus (Callebaut),Belgium,2006,Blend,West Africa,73%,"5- B,S,C,V,L","non descript, poor aftertaste",2.00
3,15,Scharffen Berger,U.S.A.,2006,Blend,Bittersweet,70%,"5- B,S,C,V,L","cherry, mild bitter",3.50
4,15,Scharffen Berger,U.S.A.,2006,Blend,Extra Dark,82%,"5- B,S,C,V,L","dry, bitter, poor aftertaste",2.00
...,...,...,...,...,...,...,...,...,...,...
2357,995,Bahen & Co.,Australia,2012,Madagascar,Sambirano,70%,"2- B,S","unrefined, flat, grassy",3.00
2358,999,Mast Brothers,U.S.A.,2012,Venezuela,Chuao,73%,"2- B,S","gritty, sour, sweet",2.75
2359,999,Mast Brothers,U.S.A.,2012,Blend,Madagascar,72%,"2- B,S","gritty, overly tart, sour",2.50
2360,999,Bahen & Co.,Australia,2012,Brazil,Bahia,70%,"2- B,S","chalky, intense, chemical",2.50


In [4]:
df.dtypes

REF                                   int64
Company (Manufacturer)               object
Company Location                     object
Review Date                           int64
Country of Bean Origin               object
Specific Bean Origin or Bar Name     object
Cocoa Percent                        object
Ingredients                          object
Most Memorable Characteristics       object
Rating                              float64
dtype: object

In [5]:
df['Cocoa Percent'] = df['Cocoa Percent'].str.rstrip('%').astype('float') / 100.0

In [6]:
df = df.dropna()

In [7]:
# Target
y = df['Rating']
# Features
X = df.drop(['Rating','Company (Manufacturer)','Company Location','Country of Bean Origin','Specific Bean Origin or Bar Name','Ingredients','Most Memorable Characteristics'], axis=1)

In [8]:
df = df.drop(columns = ('Specific Bean Origin or Bar Name'))


In [9]:
df

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,5,Jacque Torres,U.S.A.,2006,Ghana,0.71,"5- B,S,C,V,L","gritty, unrefined, off notes",2.00
1,15,Neuhaus (Callebaut),Belgium,2006,Sao Tome,0.75,"5- B,S,C,V,L","grassy, earthy, burnt",2.75
2,15,Neuhaus (Callebaut),Belgium,2006,Blend,0.73,"5- B,S,C,V,L","non descript, poor aftertaste",2.00
3,15,Scharffen Berger,U.S.A.,2006,Blend,0.70,"5- B,S,C,V,L","cherry, mild bitter",3.50
4,15,Scharffen Berger,U.S.A.,2006,Blend,0.82,"5- B,S,C,V,L","dry, bitter, poor aftertaste",2.00
...,...,...,...,...,...,...,...,...,...
2357,995,Bahen & Co.,Australia,2012,Madagascar,0.70,"2- B,S","unrefined, flat, grassy",3.00
2358,999,Mast Brothers,U.S.A.,2012,Venezuela,0.73,"2- B,S","gritty, sour, sweet",2.75
2359,999,Mast Brothers,U.S.A.,2012,Blend,0.72,"2- B,S","gritty, overly tart, sour",2.50
2360,999,Bahen & Co.,Australia,2012,Brazil,0.70,"2- B,S","chalky, intense, chemical",2.50


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=2021)

In [11]:
base_lr = LinearRegression()
base_lr.fit(X_train,y_train)

kcross_base = cross_val_score(base_lr,X_train,y_train,cv=5)


kcross_base

base_lr.score(X,y)


0.010836244263170425

In [12]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    original_dataframe = original_dataframe.append(res)
    return(original_dataframe)

In [13]:
dummies = pd.get_dummies(df[['Company Location']])
res = pd.concat([df, dummies], axis=1)
res = res.drop(['Company Location'], axis=1)
df = res
df

,REF,Company (Manufacturer),Review Date,Country of Bean Origin,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,Company Location_Amsterdam,Company Location_Argentina,...,Company Location_Switzerland,Company Location_Taiwan,Company Location_Thailand,Company Location_U.A.E.,Company Location_U.K.,Company Location_U.S.A.,Company Location_Vanuatu,Company Location_Venezuela,Company Location_Vietnam,Company Location_Wales
0,5,Jacque Torres,2006,Ghana,0.71,"5- B,S,C,V,L","gritty, unrefined, off notes",2.00,0,0,...,0,0,0,0,0,1,0,0,0,0
1,15,Neuhaus (Callebaut),2006,Sao Tome,0.75,"5- B,S,C,V,L","grassy, earthy, burnt",2.75,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,Neuhaus (Callebaut),2006,Blend,0.73,"5- B,S,C,V,L","non descript, poor aftertaste",2.00,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,Scharffen Berger,2006,Blend,0.70,"5- B,S,C,V,L","cherry, mild bitter",3.50,0,0,...,0,0,0,0,0,1,0,0,0,0
4,15,Scharffen Berger,2006,Blend,0.82,"5- B,S,C,V,L","dry, bitter, poor aftertaste",2.00,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,995,Bahen & Co.,2012,Madagascar,0.70,"2- B,S","unrefined, flat, grassy",3.00,0,0,...,0,0,0,0,0,0,0,0,0,0
2358,999,Mast Brothers,2012,Venezuela,0.73,"2- B,S","gritty, sour, sweet",2.75,0,0,...,0,0,0,0,0,1,0,0,0,0
2359,999,Mast Brothers,2012,Blend,0.72,"2- B,S","gritty, overly tart, sour",2.50,0,0,...,0,0,0,0,0,1,0,0,0,0
2360,999,Bahen & Co.,2012,Brazil,0.70,"2- B,S","chalky, intense, chemical",2.50,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df

,REF,Company (Manufacturer),Review Date,Country of Bean Origin,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,Company Location_Amsterdam,Company Location_Argentina,...,Company Location_Switzerland,Company Location_Taiwan,Company Location_Thailand,Company Location_U.A.E.,Company Location_U.K.,Company Location_U.S.A.,Company Location_Vanuatu,Company Location_Venezuela,Company Location_Vietnam,Company Location_Wales
0,5,Jacque Torres,2006,Ghana,0.71,"5- B,S,C,V,L","gritty, unrefined, off notes",2.00,0,0,...,0,0,0,0,0,1,0,0,0,0
1,15,Neuhaus (Callebaut),2006,Sao Tome,0.75,"5- B,S,C,V,L","grassy, earthy, burnt",2.75,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,Neuhaus (Callebaut),2006,Blend,0.73,"5- B,S,C,V,L","non descript, poor aftertaste",2.00,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,Scharffen Berger,2006,Blend,0.70,"5- B,S,C,V,L","cherry, mild bitter",3.50,0,0,...,0,0,0,0,0,1,0,0,0,0
4,15,Scharffen Berger,2006,Blend,0.82,"5- B,S,C,V,L","dry, bitter, poor aftertaste",2.00,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,995,Bahen & Co.,2012,Madagascar,0.70,"2- B,S","unrefined, flat, grassy",3.00,0,0,...,0,0,0,0,0,0,0,0,0,0
2358,999,Mast Brothers,2012,Venezuela,0.73,"2- B,S","gritty, sour, sweet",2.75,0,0,...,0,0,0,0,0,1,0,0,0,0
2359,999,Mast Brothers,2012,Blend,0.72,"2- B,S","gritty, overly tart, sour",2.50,0,0,...,0,0,0,0,0,1,0,0,0,0
2360,999,Bahen & Co.,2012,Brazil,0.70,"2- B,S","chalky, intense, chemical",2.50,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
dummies = pd.get_dummies(df[['Company (Manufacturer)']])
res = pd.concat([df, dummies], axis=1)
res = res.drop(['Company (Manufacturer)'], axis=1)
df = res
df

,REF,Review Date,Country of Bean Origin,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,Company Location_Amsterdam,Company Location_Argentina,Company Location_Australia,...,Company (Manufacturer)_Zak's,Company (Manufacturer)_Zart Pralinen,Company (Manufacturer)_Zokoko,Company (Manufacturer)_Zoto (Chocolatoa),Company (Manufacturer)_Zotter,Company (Manufacturer)_hello cocoa (now Markham & Fitz),Company (Manufacturer)_hexx,Company (Manufacturer)_iQ Chocolate,Company (Manufacturer)_organicfair,Company (Manufacturer)_twenty-four blackbirds
0,5,2006,Ghana,0.71,"5- B,S,C,V,L","gritty, unrefined, off notes",2.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,2006,Sao Tome,0.75,"5- B,S,C,V,L","grassy, earthy, burnt",2.75,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,2006,Blend,0.73,"5- B,S,C,V,L","non descript, poor aftertaste",2.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,2006,Blend,0.70,"5- B,S,C,V,L","cherry, mild bitter",3.50,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,2006,Blend,0.82,"5- B,S,C,V,L","dry, bitter, poor aftertaste",2.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,995,2012,Madagascar,0.70,"2- B,S","unrefined, flat, grassy",3.00,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2358,999,2012,Venezuela,0.73,"2- B,S","gritty, sour, sweet",2.75,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2359,999,2012,Blend,0.72,"2- B,S","gritty, overly tart, sour",2.50,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2360,999,2012,Brazil,0.70,"2- B,S","chalky, intense, chemical",2.50,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:
dummies = pd.get_dummies(df[['Country of Bean Origin']])
res = pd.concat([df, dummies], axis=1)
res = res.drop(['Country of Bean Origin'], axis=1)
df = res
df

,REF,Review Date,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating,Company Location_Amsterdam,Company Location_Argentina,Company Location_Australia,Company Location_Austria,...,Country of Bean Origin_Tanzania,Country of Bean Origin_Thailand,Country of Bean Origin_Tobago,Country of Bean Origin_Togo,Country of Bean Origin_Trinidad,Country of Bean Origin_U.S.A.,Country of Bean Origin_Uganda,Country of Bean Origin_Vanuatu,Country of Bean Origin_Venezuela,Country of Bean Origin_Vietnam
0,5,2006,0.71,"5- B,S,C,V,L","gritty, unrefined, off notes",2.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,2006,0.75,"5- B,S,C,V,L","grassy, earthy, burnt",2.75,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,2006,0.73,"5- B,S,C,V,L","non descript, poor aftertaste",2.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,2006,0.70,"5- B,S,C,V,L","cherry, mild bitter",3.50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,2006,0.82,"5- B,S,C,V,L","dry, bitter, poor aftertaste",2.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,995,2012,0.70,"2- B,S","unrefined, flat, grassy",3.00,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2358,999,2012,0.73,"2- B,S","gritty, sour, sweet",2.75,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2359,999,2012,0.72,"2- B,S","gritty, overly tart, sour",2.50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2360,999,2012,0.70,"2- B,S","chalky, intense, chemical",2.50,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y = df['Rating']

X = df.drop(['Rating','Ingredients','Most Memorable Characteristics'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21)

base_lr = LinearRegression()
base_lr.fit(X_train,y_train)

kcross_base = cross_val_score(base_lr,X_train,y_train,cv=5)

kcross_base

base_lr.score(X,y)

-7078032967237206.0

In [24]:
dt = DecisionTreeClassifier(random_state=2021)

In [28]:
dt.fit(X_train, y_train)

# make predictions on training and validation data
train_preds = dt.predict(X_train)


# Print accuracy score
print('Training accuracy: ', accuracy_score(y_train, train_preds))

ValueError: Unknown label type: 'continuous'